## Multiple object tracking with YOLOv3-based object detection

In [8]:
import numpy as np
import cv2 as cv
from motrackers.detectors import nanodet
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
from nanodet.util import Logger, cfg, load_config, load_model_weight
import ipywidgets as widgets

In [9]:
VIDEO_FILE = r"D:\shijue\LiquidDrop\22.avi"
WEIGHTS_PATH = r'D:\shijue\multi-object-tracker\weight\LiquidV4.pth'
CONFIG_FILE_PATH = r'D:\shijue\multi-object-tracker\config\LiquidDetect416.yml'

In [10]:
chosen_tracker = widgets.Select(
    options=["CentroidTracker", "CentroidKF_Tracker", "SORT", "IOUTracker"],
    value='CentroidTracker',
    rows=5,
    description='MOTracker:',
    disabled=False
)
chosen_tracker

Select(description='MOTracker:', options=('CentroidTracker', 'CentroidKF_Tracker', 'SORT', 'IOUTracker'), valu…

In [11]:
if chosen_tracker.value == 'CentroidTracker':
    tracker = CentroidTracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'CentroidKF_Tracker':
    tracker = CentroidKF_Tracker(max_lost=0, tracker_output_format='mot_challenge')
elif chosen_tracker.value == 'SORT':
    tracker = SORT(max_lost=3, tracker_output_format='mot_challenge', iou_threshold=0.3)
elif chosen_tracker.value == 'IOUTracker':
    tracker = IOUTracker(max_lost=2, iou_threshold=0.5, min_detection_confidence=0.4, max_detection_confidence=0.7,
                         tracker_output_format='mot_challenge')
else:
    print("Please choose one tracker from the above list.")

In [12]:
# 导入模型文件
local_rank = 0
model = WEIGHTS_PATH
device = "cpu:0"
config = CONFIG_FILE_PATH
logger = Logger(local_rank, use_tensorboard=False)
load_config(cfg, config)
detmodel = nanodet.Nanodet(cfg, model, logger, device)
logger.log('Press "Esc", "q" or "Q" to exit.')

model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


[root][04-10 20:46:36]INFO:Press "Esc", "q" or "Q" to exit.
[root][04-10 20:46:36]INFO:Press "Esc", "q" or "Q" to exit.


In [13]:
def main(video_path, model, tracker):

    cap = cv.VideoCapture(video_path)
    while True:
        ok, image = cap.read()

        if not ok:
            print("Cannot read the video feed.")
            break
        
        meta, res = detmodel.inference(image)
        bboxes, confidences, class_ids = detmodel.visualize(res[0], meta, cfg.class_names, 0.43)
        
        tracks = tracker.update(bboxes, confidences, class_ids)
        
        updated_image = detmodel.draw_bboxes(image.copy(), bboxes, confidences, class_ids)

        updated_image = draw_tracks(updated_image, tracks)

        cv.imshow("image", updated_image)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

In [14]:
main(VIDEO_FILE, model, tracker)

forward time: 0.146s | decode time: 0.004s | viz time: 0.000s


AttributeError: 'Nanodet' object has no attribute 'draw_bboxes'